In [15]:
import pandas as pd
import numpy as np

### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:  
- оценка 2 и меньше - низкий рейтинг  
- оценка 4 и меньше - средний рейтинг  
- оценка 4.5 и 5 - высокий рейтинг  

Результат классификации запишите в столбец class



In [16]:
def class_name(rating):
    if rating <= 2:
        return('низкий рейтинг')
    if rating <= 4:
        return('средний рейтинг')
    return('высокий рейтинг')

data_rating = pd.read_csv('ml-latest-small/ratings.csv')
data_rating['class'] = data_rating['rating'].apply(class_name)
data_rating.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data =  {
            'Центр': ['москва', 'тула', 'ярославль'],  
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],  
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  
            }
  
Результат классификации запишите в отдельный столбец region.

In [17]:

def region(raw, geo_):
    for reg in geo_.keys():
        for city in geo_[reg]:
            if city in raw['keyword']:
                return reg
    return 'undefined'

geo_data = { 'Центр': ['москва', 'тула', 'ярославль'],
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
           }
data_rating = pd.read_csv('ml-latest-small/keywords.csv')
data_rating['region'] = data_rating.apply(region, geo_ = geo_data, axis = 1)
data_rating.head(130)

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
125,школьный портал,982692,undefined
126,вк моя страница,979105,undefined
127,авито москва,979292,Центр
128,спортмастер,968154,undefined


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years  
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска  
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год  

Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [47]:
import re
def production_year(name, ys):
    try:
        y = int(re.findall(r'\((\d{4})\)',name)[0])
    except:
        return 1900
    if y in ys:
        return y
    return 1900

years = np.arange(1950,2010).tolist()
data_movie = pd.read_csv('ml-latest-small/movies.csv')
data_movie['year'] = data_movie['title'].apply(production_year, ys = years)
data_rating = pd.read_csv('ml-latest-small/ratings.csv')
data_rating = data_rating.groupby('movieId').agg({'rating': 'mean'})
data = ((pd.merge(data_movie, data_rating, how ='inner', on ='movieId'))
        .groupby('year')
        .agg({'rating': 'mean'})
        .sort_values(by='rating', ascending=False))
data.head(30)

,rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
1973,3.688212
1971,3.643227
1969,3.604509
1972,3.585295
